# 自定义入门

[![下载Notebook](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_common.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_common.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/network/common.ipynb)

MindSpore向用户提供了高阶、中阶和低阶3个不同层次的API，详细内容参见[基本介绍-层次结构内容章节](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/introduction.html#层次结构)。

为方便控制网络的执行流程，MindSpore提供了高阶的训练和推理接口`mindspore.Model`，通过指定要训练的神经网络模型和常见的训练设置，调用`train`和`eval`方法对网络进行训练和推理。同时，用户如果想要对特定模块进行个性化设置，也可以调用对应的中低阶接口自行定义网络的训练流程。

初始化网络之前，需要配置`context`参数，用于控制程序执行的策略，如配置静态图或动态图模式，配置网络运行的硬件环境等。本章将会介绍配置信息和使用MindSpore提供的中低阶API自定义损失函数、优化器、训练流程、Metric、自定义验证流程模块。

## 配置信息

初始化网络之前要配置`context`参数，用于控制程序执行的策略，本节主要介绍执行模式管理和硬件管理。

### 执行模式

MindSpore支持Graph和PyNative两种运行模式。Graph模式是MindSpore的默认模式，而PyNative模式用于调试等用途。

- Graph模式（静态图模式）：将神经网络模型编译成一整张图，然后下发到硬件执行。该模式利用图优化等技术提高运行性能，同时有助于规模部署和跨平台运行。

- PyNative模式（动态图模式）：将神经网络中的各个算子逐一下发到硬件中执行，该模式方便用户编写代码和调试神经网络模型。

#### 模式选择

通过配置context参数可以控制程序运行的模式。Graph和PyNative两种模式的区别主要有：

- 使用场景：Graph模式需要一开始就构建好网络结构，然后框架做整图优化和执行，比较适合网络固定没有变化，且需要高性能的场景。而PyNative模式逐行执行算子，支持执行单算子、普通函数和网络，以及单独求梯度的操作。

- 网络执行：Graph模式和PyNative模式在执行相同的网络和算子时，精度效果是一致的。由于Graph模式运用了图优化、计算图整图下沉等技术，Graph模式执行网络的性能和效率更高。

- 代码调试：在脚本开发和网络流程调试中，推荐使用PyNative模式进行调试。在PyNative模式下，可以方便地设置断点，获取网络执行的中间结果，也可以通过pdb的方式对网络进行调试。而Graph模式无法设置断点，只能先指定算子进行打印，然后在网络执行完成后查看输出结果。

使用Graph模式时，将context中的运行模式设置为`GRAPH_MODE`，需要使用`nn.Cell`类，并在`construct`函数中编写执行代码，或者调用`@ms_function`装饰器。

#### 模式切换

MindSpore提供了静态图和动态图统一的编码方式，大大增加了静态图和动态图的可兼容性，用户无需开发多套代码，仅变更一行代码便可切换静态图/动态图模式。模式切换时，请留意目标模式的[约束](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)。

> 例如，PyNative模式不支持数据下沉等。

首先定义网络模型`MyNet`和后续代码片段用到的数据，用于后续的动静态图模式的切换和展示。

In [17]:
import numpy as np

import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor

class MyNet(nn.Cell):
    def __init__(self):
        super(MyNet, self).__init__()
        self.mul = ops.Mul()

    def construct(self, x, y):
        return self.mul(x, y)

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

设置运行模式为静态图模式：

In [18]:
from mindspore import context

context.set_context(mode=context.GRAPH_MODE)

net = MyNet()
print(net(x, y))

[ 4. 10. 18.]


MindSpore处于静态图模式时，可以通过 `context.set_context(mode=context.PYNATIVE_MODE)`切换为动态图模式；同样，MindSpore处于动态图模式时，可以通过`context.set_context(mode=context.GRAPH_MODE)`切换为静态图模式。

In [3]:
context.set_context(mode=context.PYNATIVE_MODE)

net = MyNet()
print(net(x, y))

[ 4. 10. 18.]


### 硬件管理

硬件管理部分主要包括`device_target`和`device_id`两个参数。

- `device_target`： 待运行的目标设备，支持`Ascend`、`GPU`和`CPU`，可以根据实际环境情况设置。

- `device_id`： 表示目标设备ID，其值在[0, `device_num_per_host` - 1]范围内，`device_num_per_host`表示服务器的总设备数量，`device_num_per_host`的值不能超过4096，`device_id`默认为0。在非分布式模式执行的情况下，为了避免设备的使用冲突，可以通过设置`device_id`决定程序执行的设备ID。

代码样例如下：

```Python
from mindspore import context

context.set_context(device_target="Ascend", device_id=6)
```

## 定义数据集

MindSpore的`mindspore.dataset`模块集成了常见的数据处理功能：用户既可以调用此模块的相关接口来[加载常见的数据集](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.dataset.html)，也可以构造数据集类并使用`mindspore.dataset.GeneratorDataset`接口实现自定义数据集加载。

如下示例定义`get_data`函数生成样本数据及对应的标签，定义`create_dataset`函数加载自定义数据集。

In [4]:
import numpy as np
import mindspore.dataset as ds

def get_data(data_num, data_size):
    """生成样本数据及对应的标签"""
    for _ in range(data_num):
        data = np.random.randn(data_size)
        p = np.array([1, 0, -3, 5])
        label = np.polyval(p, data).sum()
        yield data.astype(np.float32), np.array([label]).astype(np.float32)

def create_dataset(data_num, data_size, batch_size=32, repeat_size=1):
    """定义数据集"""
    input_data = ds.GeneratorDataset(list(get_data(data_num, data_size)), column_names=['data', 'label'])
    input_data = input_data.batch(batch_size)
    input_data = input_data.repeat(repeat_size)
    return input_data

## 定义网络

mindspore.nn`类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承`nn.Cell`类，并重写`__init__`方法和`construct`方法。`mindspore.ops`模块提供了基础算子的实现，`nn.Cell`模块实现了对基础算子的进一步封装，用户可以根据需要，灵活使用不同的算子。

如下示例使用`nn.Cell`构建一个简单的全连接网络，用于后续自定义内容的示例片段代码。

In [5]:
from mindspore import nn
import mindspore.common.initializer as init

class MyNet(nn.Cell):
    """定义全连接网络"""
    def __init__(self, input_size=32):
        super(MyNet, self).__init__()
        self.fc1 = nn.Dense(input_size, 120, weight_init=init.Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=init.Normal(0.02))
        self.fc3 = nn.Dense(84, 1, weight_init=init.Normal(0.02))
        self.relu = nn.ReLU()

    def construct(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 自定义损失函数

损失函数（Loss Function）用于衡量预测值与真实值差异的程度。深度学习中，模型训练就是通过不停地迭代来缩小损失函数值的过程，因此在模型训练过程中损失函数的选择非常重要，定义一个好的损失函数可以帮助损失函数值更快收敛，达到更好的精度。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id13)提供了许多通用损失函数供用户选择， 也支持用户根据需要自定义损失函数。

自定义损失函数类时，既可以继承网络的基类`nn.Cell`，也可以继承损失函数的基类`nn.LossBase`。`nn.LossBase`在`nn.Cell`的基础上，提供了`get_loss`方法，利用`reduction`参数对损失值求和或求均值，输出一个标量。下面将分别使用继承Cell和继承LossBase的方法来定义平均绝对误差损失函数(Mean Absolute Error，MAE)，MAE算法的公式如下所示：

$$ loss= \frac{1}{m}\sum_{i=1}^m\lvert y_i-f(x_i) \rvert $$

上式中$f(x)$为预测值，$y$为样本真实值，$loss$为预测值与真实值之间距离的平均值。

### 继承Cell定义损失函数

`nn.Cell`既可以用于定义网络，也可以定义损失函数。使用`nn.Cell`定义损失函数与定义一个普通的网络的差别在于，其执行逻辑用于计算前向网络输出与真实值之间的误差。

示例代码如下：

In [6]:
from mindspore import nn, ops

class MyMAELoss1(nn.Cell):
    """定义损失"""
    def __init__(self):
        super(MyMAELoss1, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()

    def construct(self, predict, target):
        x = self.abs(predict - target)
        return self.reduce_mean(x)

### 继承LossBase定义损失函数

`nn.LossBase`作为损失函数的基类，使用其自定义损失函数时，需要重写`__init__`方法和`construct`方法，使用`get_loss`方法计算损失。示例代码如下：

In [7]:
from mindspore import nn, ops

class MyMAELoss2(nn.LossBase):
    """定义损失"""
    def __init__(self, reduction="mean"):
        super(MyMAELoss2, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, predict, target):
        x = self.abs(predict - target)
        return self.get_loss(x)

## 自定义优化器

优化器在模型训练过程中，用于计算和更新网络参数，合适的优化器可以有效减少训练时间，提高模型性能。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id14)提供了许多通用的优化器供用户选择，同时也支持用户根据需要自定义优化器。

自定义优化器时可以继承优化器基类`nn.Optimizer`，重写`__init__`方法和`construct`方法实现参数的更新。

如下示例实现自定义优化器Momentum：

$$ v_{t+1} = v_t×u+grad $$

带有动量的SGD算法：

$$p_{t+1} = p_t - lr*v_{t+1}$$

使用Nesterov动量的SGD算法:

$$p_{t+1} = p_t-(grad+v_{t+1}*u)×lr $$

其中，grad 、lr 、p 、v 和 u 分别表示梯度、学习率、参数、矩（Moment）和动量（Momentum）。

In [8]:
from mindspore import Tensor, Parameter
from mindspore import nn, ops
from mindspore import dtype as mstype

class MyMomentum(nn.Optimizer):
    """定义优化器"""
    def __init__(self, params, learning_rate, momentum=0.9, use_nesterov=False):
        super(MyMomentum, self).__init__(learning_rate, params)
        self.momentum = Parameter(Tensor(momentum, mstype.float32), name="momentum")
        self.use_nesterov = use_nesterov
        self.moments = self.parameters.clone(prefix="moments", init="zeros")
        self.assign = ops.Assign()

    def construct(self, gradients):
        """construct输入为梯度，在训练中自动传入梯度gradients"""
        lr = self.get_lr()
        # 待更新的权重参数
        params = self.parameters
        for i in range(len(params)):
            # 更新moments值
            self.assign(self.moments[i], self.moments[i] * self.momentum + gradients[i])
            if self.use_nesterov:
                # 使用Nesterov动量的SGD算法:
                update = params[i] - (self.moments[i] * self.momentum + gradients[i]) * lr
            else:
                # 带有动量的SGD算法
                update = params[i] - self.moments[i] * lr
            # 更新params值为update值
            self.assign(params[i], update)
        return params

`MindSpore.ops`也封装了优化器算子供用户自行定义优化器，如`ops.ApplyCenteredRMSProp`、 `ops.ApplyMomentum`和`ops.ApplyRMSProp`等。如下示例使用`ApplyMomentum`算子自定义优化器Momentum：

In [9]:
class MyMomentum2(nn.Optimizer):
    """定义优化器"""
    def __init__(self, params, learning_rate, momentum=0.9, use_nesterov=False):
        super(MyMomentum2, self).__init__(learning_rate, params)
        self.moments = self.parameters.clone(prefix="moments", init="zeros")
        self.momentum = momentum
        self.opt = ops.ApplyMomentum(use_nesterov=use_nesterov)

    def construct(self, gradients):
        params = self.parameters
        success = None
        for param, mom, grad in zip(params, self.moments, gradients):
            success = self.opt(param, mom, self.learning_rate, grad, self.momentum)
        return success

## 自定义训练流程

`mindspore.nn`模块提供了`train`和`eval`训练和推理的接口方便用户在训练过程中使用，此外用户还可以自行定义训练过程。!!!wushuiqin,补充好下面这句话，解释清楚为什么要定义损失函数，不懂问问小贤：自定以训练过程需要首先定义损失函数，然后把损失函数和训练流程结合起来。

### 定义损失网络

定义损失网络`MyWithLossCell`，将前向网络与损失函数连接起来。

In [10]:
class MyWithLossCell(nn.Cell):
    """定义损失网络"""

    def __init__(self, backbone, loss_fn):
        """实例化时传入前向网络和损失函数作为参数"""
        super(MyWithLossCell, self).__init__(auto_prefix=False)
        self.backbone = backbone
        self.loss_fn = loss_fn

    def construct(self, data, label):
        """连接前向网络和损失函数"""
        out = self.backbone(data)
        return self.loss_fn(out, label)

    def backbone_network(self):
        """要封装的骨干网络"""
        return self.backbone

### 定义训练流程

定义训练流程`MyTrainStep`，该类继承`nn.TrainOneStepCell`，`nn.TrainOneStepCell`封装了损失网络和优化器，在执行训练时通过`ops.GradOperation`算子来进行梯度的获取，通过优化器来实现权重的更新。

In [11]:
class MyTrainStep(nn.TrainOneStepCell):
    """定义训练流程"""

    def __init__(self, network, optimizer):
        """参数初始化"""
        super(MyTrainStep, self).__init__(network, optimizer)
        self.grad = ops.GradOperation(get_by_list=True)

    def construct(self, data, label):
        """构建训练过程"""
        weights = self.weights
        loss = self.network(data, label)
        grads = self.grad(self.network, weights)(data, label)
        return loss, self.optimizer(grads)

### 执行训练

执行训练，并打印输出每个Epoch的损失值。

In [ ]:
# 生成多项式分布的训练数据
dataset_size = 32
ds_train = create_dataset(2048, dataset_size)

# 定义网络
net = MyNet()
# 损失函数
loss_func = MyMAELoss1()
# 优化器
opt = MyMomentum(net.trainable_params(), 0.01)
# 构建损失网络

net_with_criterion = MyWithLossCell(net, loss_func)
# 构建训练网络
train_net = MyTrainStep(net_with_criterion, opt)

# 执行训练，每个epoch打印一次损失值
epochs = 5
for i in range(epochs):
    for train_x, train_y in ds_train:
        train_net(train_x, train_y)
        loss_val = net_with_criterion(train_x, train_y)
    print(f"Epoch:[{i+1}/{epochs}], loss:{loss_val}")

```Text
Epoch:[1/5], loss:101.55868
Epoch:[2/5], loss:15.018886
Epoch:[3/5], loss:14.187622
Epoch:[4/5], loss:29.534908
Epoch:[5/5], loss:13.403777
```

## 自定义评价指标

当训练任务结束，常常需要评价指标（Metrics）评估函数来评估模型的好坏。常用的评价指标混淆矩阵、准确率 Accuracy、精确率 Precision、召回率 Recall等。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id16)模块提供了常见的评估函数，用户也可以根据需要自行定义评估指标。自定义Metrics函数需要继承`nn.Metric`父类，并重新实现父类中的`clear`方法、`update`方法和`eval`方法。平均绝对误差（MAE）算法如下式所示，下面以简单的MAE为例，介绍这三个函数及其使用方法。

$$ MAE=\frac{1}{n}\sum_{i=1}^n\lvert ypred_i - y_i \rvert$$

- `clear`：初始化相关的内部参数。
- `update`：接收网络预测输出和标签，计算误差，并更新内部评估结果。一般在每个step后进行计算，并更新统计值。
- `eval`：计算最终评估结果，一般在一个epoch结束后计算最终的评估结果。

In [13]:
class MyMAE(nn.Metric):
    """定义metric"""

    def __init__(self):
        super(MyMAE, self).__init__()
        self.clear()

    def clear(self):
        """初始化变量abs_error_sum和samples_num"""
        self.abs_error_sum = 0
        self.samples_num = 0

    def update(self, *inputs):
        """更新abs_error_sum和samples_num"""
        if len(inputs) != 2:
            raise ValueError('Mean absolute error need 2 inputs (y_pred, y), but got {}'.format(len(inputs)))
        # 将Tensor转换为NumPy，便于后续计算
        y_pred = inputs[0].asnumpy()
        y = inputs[1].asnumpy()
        # 计算预测值与真实值的绝对误差
        error_abs = np.abs(y.reshape(y_pred.shape) - y_pred)
        self.abs_error_sum += error_abs.sum()
        # 样本的总数
        self.samples_num += y.shape[0]

    def eval(self):
        """计算最终评估结果"""
        if self.samples_num == 0:
            raise RuntimeError('Total samples num must not be 0.')
        return self.abs_error_sum / self.samples_num

## 自定义验证流程

mindspore.nn模块提供了评估网络包装函数[nn.WithEvalCell](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/nn/mindspore.nn.WithEvalCell.html#mindspore.nn.WithEvalCell)，由于`nn.WithEvalCell`只有两个输入`data`和`label`，不适用于多数据或多标签的场景，所以需要自定义评估网络。多标签场景下自定义评估网络可参考[自定义评估与训练章节](https://www.mindspore.cn/tutorials/zh-CN/master/advance/network/custom_train_and_eval_net.html#自定义评估网络)。

如下示例实现简单的自定义评估网络`MyWithEvalCell`，输入传入数据`data`和标签`label`:

In [14]:
class MyWithEvalCell(nn.Cell):
    """定义验证流程"""

    def __init__(self, network):
        super(MyWithEvalCell, self).__init__(auto_prefix=False)
        self.network = network

    def construct(self, data, label):
        outputs = self.network(data)
        return outputs, label

执行推理并评估：


In [15]:
# 获取验证数据
ds_eval = create_dataset(128, dataset_size, 1)
# 定义评估网络
eval_net = MyWithEvalCell(net)
eval_net.set_train(False)
# 定义评估指标
mae = MyMAE()

# 执行推理过程
for eval_x, eval_y in ds_eval:
    output, eval_y = eval_net(eval_x, eval_y)
    mae.update(output, eval_y)

mae_result = mae.eval()
print("MAE: ", mae_result)

MAE:  17.830261886119843


输出评估误差，MAE与模型在训练集上效果大致相同。